In [6]:
"""
Module for performing parameter fitting for synthetic datasets 
(parameter recovery analysis)

Author: Alvaro Garrido Perez <alvaro.garridoperez@ugent.be>
Date: 09-12-2025

"""




"""-------------IMPORT PACKAGES-------------"""

import numpy as np 
import csv 
import glob
import pandas as pd
import os
import sys
from tqdm import tqdm
from joblib import Parallel, delayed
import ast

# Get the path to the parent directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the Python path
sys.path.append(parent_dir)

current_dir = os.getcwd()


# Import utils folder
from utils.twostep_support import *    
from models import *
from MLE import *

"""---------------SELECT MODEL AND SET VARIABLES ---------------"""

model = "AI_ddm" # RL, RL_ddm, AI, or AI_ddm
mtype = 1 # 0, 1, 2 or 3 (only relevant if model = AI or AI_ddm)
drmtype = "linear" # Drift rate model: linear, sigmoid, sigmoid_single_v_mod, sigmoid_single_v_max
n_starts = 2 # Number of random starting parameter values in parameter fitting.
dataset = 'magic_carpet_2020' 
optimizer = 'L-BFGS-B' 

if model in ("RL", "RL_ddm"):
    learning = "RL"
elif model in ("AI", "AI_ddm"):
    learning = "PSM"
    

"""-------LOAD SYNTHETIC DATA AND PARAMETERS-------"""

# Path to dataset folder
current_dir = os.getcwd()

if model == "RL":
    path_to_folder = (os.path.join(current_dir, 'param_recovery_data/' + dataset + '/model_' + model +'/'))
    file_path_synthetic_data = os.path.join(path_to_folder, "param_recovery_synthetic_data_M" + model + ".csv")
    file_path_synthetic_params = os.path.join(path_to_folder, "param_recovery_synthetic_params_M" + model + ".csv")

elif model == "RL_ddm":
    path_to_folder = (os.path.join(current_dir, 'param_recovery_data/' + dataset + '/model_' + model + "_DRM" + drmtype + '/'))
    file_path_synthetic_data = os.path.join(path_to_folder, "param_recovery_synthetic_data_M" + model + "_DRM" + drmtype + ".csv")
    file_path_synthetic_params = os.path.join(path_to_folder, "param_recovery_synthetic_params_M" + model + "_DRM" + drmtype +".csv")
    
    
elif model == "AI":
    path_to_folder = (os.path.join(current_dir, 'param_recovery_data/' + dataset + '/model_' + model + str(mtype) + '/'))
    file_path_synthetic_data = os.path.join(path_to_folder, "param_recovery_synthetic_data_M" + model + str(mtype) + ".csv")
    file_path_synthetic_params = os.path.join(path_to_folder, "param_recovery_synthetic_params_M" + model + str(mtype) + ".csv")

elif model == "AI_ddm":    
    path_to_folder = (os.path.join(current_dir, 'param_recovery_data/' + dataset + '/model_' + model + str(mtype) + "_DRM" + drmtype + '/'))
    file_path_synthetic_data = os.path.join(path_to_folder, "param_recovery_synthetic_data_M" + model + str(mtype) + "_DRM" + drmtype + ".csv")
    file_path_synthetic_params = os.path.join(path_to_folder, "param_recovery_synthetic_params_M" + model + str(mtype) + "_DRM" + drmtype +".csv")

df_synthetic_data = pd.read_csv(file_path_synthetic_data)

#Get number of trials T
df_par = df_synthetic_data[df_synthetic_data['Synthetic_participant_ID'] == 1]
T = len(ast.literal_eval(df_par['choice1'].iloc[0])) #Number of trials

synthetic_par_ID_list = np.unique(df_synthetic_data['Synthetic_participant_ID'])

df_synthetic_params = pd.read_csv(file_path_synthetic_params)

"""-------------------DEFINE MODEL PARAMETERS---------------------"""

# Different model classes and parameter settings
if model == "RL":
    p_names = ["lr1", "lr2", "lam", "b1", "b2", "p", "w"]
    lower_bounds = np.array([0, 0, 0, 0, 0, -1, 0])
    upper_bounds = np.array([1, 1, 1, 20, 20, 1, 1])

elif model == "RL_ddm":
    if drmtype == "linear":
        p_names = ["lr1", "lr2", "lam","w","p" , "a_bs", "ndt", "v_stage_0", "v_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10])
    elif drmtype == "sigmoid_single_v_mod":
        p_names = ["lr1", "lr2", "lam", "w","p","a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10, 1])
    elif drmtype == "sigmoid":
        p_names = ["lr1", "lr2", "lam", "w","p","a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod_stage_0", "v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 10, 1, 1])
    elif drmtype == "sigmoid_single_v_max":
        p_names = ["lr1", "lr2", "lam", "w","p","a_bs", "ndt", "v_max", "v_mod_stage_0", "v_mod_stage_1"]
        lower_bounds = np.array([0, 0, 0, 0, -1, 0.3, 0, 0, 0, 0])
        upper_bounds = np.array([1, 1, 1, 1, 1, 4, 1, 10, 1, 1])
        
elif model == "AI":
    if mtype == 0:
        p_names = ["lr","vunsamp", "vsamp", "vps", "gamma1", "gamma2", "lam", "kappa_a", "prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 1:
        p_names = ["lr","vunsamp", "vps", "gamma1", "gamma2", "lam", "kappa_a", "prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 2:
        p_names = ["lr", "vsamp", "vps", "gamma1", "gamma2", "lam", "kappa_a", "prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8])
    elif mtype == 3:
        p_names = ["lr","vunsamp", "vsamp", "gamma1", "gamma2", "lam", "kappa_a", "prior_r"]
        lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0, 0.2])
        upper_bounds = np.array([4, 0.9, 0.9, 20, 20, 10, 5, 0.8])     

elif model == "AI_ddm":
    if mtype == 0:
        if drmtype == "linear":
            p_names = ["lr","vunsamp", "vsamp", "vps", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_stage_0", "v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])
        if drmtype == "sigmoid":
            p_names = ["lr","vunsamp", "vsamp", "vps", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1]) 
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["lr","vunsamp", "vsamp", "vps", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1]) 
        if drmtype == "sigmoid_single_v_max":
            p_names = ["lr","vunsamp", "vsamp", "vps", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])
    elif mtype == 1:
        if drmtype == "linear":
            p_names = ["lr","vunsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_stage_0", "v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])
        if drmtype == "sigmoid":
            p_names = ["lr","vunsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["lr","vunsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
        if drmtype == "sigmoid_single_v_max":
            p_names = ["lr","vunsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])
    elif mtype == 2:
        if drmtype == "linear":
            p_names = ["lr", "vsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_stage_0", "v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])

        if drmtype == "sigmoid":
            p_names = ["lr", "vsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["lr", "vsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
        if drmtype == "sigmoid_single_v_max":
            p_names = ["lr", "vsamp", "vps","lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])
        
    elif mtype == 3:
        if drmtype == "linear":
            p_names = ["lr","vunsamp", "vsamp", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_stage_0", "v_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10])

        if drmtype == "sigmoid":
            p_names = ["lr","vunsamp", "vsamp", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1, 1])
        if drmtype == "sigmoid_single_v_mod":
            p_names = ["lr","vunsamp", "vsamp", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max_stage_0", "v_max_stage_1", "v_mod"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 10, 1])
            
        if drmtype == "sigmoid_single_v_max":
            p_names = ["lr","vunsamp", "vsamp", "lam", "kappa_a", "prior_r", "a_bs", "ndt", "v_max", "v_mod_stage_0", "v_mod_stage_1"]
            lower_bounds = np.array([0, 0, 0, 0, 0, 0.2, 0.3, 0, 0, 0, 0])
            upper_bounds = np.array([4, 0.9, 0.9, 10, 5, 0.8, 4, 1, 10, 1, 1])


nump = len(p_names) # Parameter names

"""----------------DEFINE FITTING FUNCTIONS------------------"""

# Function to process each participant
def process_participant(par):
    
    df_par = df_synthetic_data[df_synthetic_data['Synthetic_participant_ID'] == par]
    df_synthetic_param_set = df_synthetic_params[df_synthetic_params['Synthetic_participant_ID'] == par]

    T = len(ast.literal_eval(df_par["choice1"].iloc[0])) 

    # Prepare data
    actions = np.zeros((T, 2))
    actions_hssm = np.zeros((T,2))
    observations = np.zeros((T, 2))
    rts = np.zeros((T, 2))

    actions[:,0] = ast.literal_eval(df_par["choice1"].iloc[0])
    actions[:,1] = ast.literal_eval(df_par["choice2"].iloc[0])
    observations[:,0] = ast.literal_eval(df_par["final_state"].iloc[0])
    observations[:,1] = ast.literal_eval(df_par["reward"].iloc[0])

    if model in ("RL_ddm","AI_ddm"):
        rts[:,0] = ast.literal_eval(df_par["rt1"].iloc[0])
        rts[:,1] = ast.literal_eval(df_par["rt2"].iloc[0])

    for i in range(len(actions[:,0])):
        if actions[i,0] == 0:
            actions_hssm[i,0] = -1
        else:
            actions_hssm[i,0] = actions[i,0]

        if actions[i,1] == 0:
            actions_hssm[i,1] = -1
        else:
            actions_hssm[i,1] = actions[i,1]

    # Perform MLE procedure
    try:
        best_p, minNLL, NLLs = MLE_procedure(params = p_names,
                                           observations = observations.astype(int),
                                           actions = actions.astype(int),
                                           actions_hssm = actions_hssm.astype(int),
                                           rts = rts,
                                           learning = learning,
                                           lower_bounds = lower_bounds,
                                           upper_bounds = upper_bounds,
                                           n_starts = n_starts,
                                           model=model,
                                           mtype=mtype,
                                           drmtype=drmtype,
                                           seed=par,
                                           optimizer = optimizer)

        best_fitted_params = best_p.tolist()

        results_array = {"ParticipantID": par, "NLL": minNLL}

        for p, param in enumerate(p_names):
            results_array["Recovered_" + param] = best_fitted_params[p] 
            results_array["Synthetic_" + param] = df_synthetic_param_set["Synthetic_" + param].iloc[0]

    
    except Exception as e:
        print(f"MLE procedure failed for participant {par}: {e}")
        results_array = {"ParticipantID": par}
        for p, param in enumerate(p_names):
            results_array["Recovered_" + param] = 0 
            results_array["Synthetic_" + param] = df_synthetic_param_set["Synthetic_" + param].iloc[0]

    return results_array


"""------------------------FIT PARAMETERS--------------------------"""

print('Starting parameter fitting of synthetic data (parameter recovery analysis)')
print(f'Model: {model}')
print(f'Dataset : {dataset}')
print(f'N starts: {n_starts}')

if model in ("AI_ddm", "AI"):
    print(f'Mtype : {mtype}')
if model in ("AI_ddm", "RL_ddm"):
    print(f'Drmtype : {drmtype}')

# Parallel execution using Joblib
num_cores = os.cpu_count()  # Detect the number of CPU cores

results = Parallel(n_jobs=10, verbose = 1)(delayed(process_participant)(par) for par in synthetic_par_ID_list)

df_param_recovery_results = pd.DataFrame(results)


"""----------------SAVE FITTED PARAMETERS---------------"""
if model == "RL":
    file_path = os.path.join(path_to_folder, "param_recovery_results_M" + model + "_n_starts" + str(n_starts) + ".csv")
elif model == "RL_ddm":
    file_path = os.path.join(path_to_folder, "param_recovery_results_M" + model + "_DRM" + drmtype + "_n_starts" + str(n_starts) + ".csv")
elif model == "AI":
    file_path = os.path.join(path_to_folder, "param_recovery_results_M" + model + str(mtype) + "_n_starts" + str(n_starts) + ".csv")
elif model == "AI_ddm":    
    file_path = os.path.join(path_to_folder, "param_recovery_results_M" + model + str(mtype) + "_DRM" + drmtype + "_n_starts" + str(n_starts) + ".csv")
    
df_param_recovery_results.to_csv(file_path, index=False)

Starting parameter fitting of synthetic data (parameter recovery analysis)
Model: AI_ddm
Dataset : magic_carpet_2020
N starts: 2
Mtype : 1
Drmtype : linear


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.17565D+01    |proj g|=  8.53975D+00
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.88000D+02    |proj g|=  7.65908D+00
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.04426D+02    |proj g|=  4.17305D+00
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           10     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.24355D+02    |proj g|=  9.31206D+00
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate  135    f= -3.98134D+00    |proj g|=  9.05744D-01

At iterate   28    f= -2.60864D+00    |proj g|=  7.10760D-01

At iterate  146    f= -2.72017D+00    |proj g|=  3.42300D-03

At iterate   80    f= -5.02258D+00    |proj g|=  5.99985D-01

At iterate  113    f= -3.65481D+00    |proj g|=  5.35657D-01

At iterate  100    f= -3.83732D+00    |proj g|=  7.05902D-01

At iterate  148    f= -1.79016D+00    |proj g|=  1.67035D-01

At iterate  137    f= -4.40690D+00    |proj g|=  5.60962D-01

At iterate   59    f= -3.59667D+00    |proj g|=  1.66897D+00

At iterate  136    f= -3.98749D+00    |proj g|=  9.05817D-01

At iterate  147    f= -2.72017D+00    |proj g|=  4.07225D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F   

...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate  122    f= -3.96554D+00    |proj g|=  8.93663D-01

At iterate  157    f= -4.04027D+00    |proj g|=  9.27680D-01

At iterate   45    f= -3.18535D+00    |proj g|=  7.35461D-01

At iterate   81    f= -3.81799D+00    |proj g|=  9.19297D-01

At iterate   99    f= -5.02359D+00    |proj g|=  9.40331D-01

At iterate   15    f= -7.27347D-02    |proj g|=  1.02687D+00

At iterate  158    f= -4.04931D+00    |proj g|=  9.05658D-01

At iterate    8    f=  7.22228D-01    |proj g|=  9.85805D-01

At iterate  123    f= -3.96556D+00    |proj g|=  8.93583D-01

At iterate   46    f= -3.32689D+00    |proj g|=  5.38676D-01

At iterate  155    f= -4.47876D+00    |proj g|=  4.38197D-01

At iterate  100    f= -5.02447D+00    |proj g|=  6.71504D-01

At iterate  171    f= -1.79241D+00    |proj g|=  4.33469D-01

At iterate  159    f= -4.05788D+00    |proj g|=  9.05708D-01

At iterate   82    f= -3.81844D+00    |proj g|=  9.19268D-01

At iterate    9    f=  6.53043D-01    |proj g|=  9.63907D-01

At iter


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   50    f= -6.74295D-01    |proj g|=  7.21783D-01

At iterate  131    f= -5.61214D+00    |proj g|=  7.56657D-01

At iterate   64    f= -2.35965D+00    |proj g|=  4.91934D-01

At iterate  186    f= -3.76530D+00    |proj g|=  2.83619D-01

At iterate   96    f= -3.51943D+00    |proj g|=  6.14026D-01

At iterate  237    f= -4.19181D+00    |proj g|=  9.05969D-01

At iterate  213    f= -1.80533D+00    |proj g|=  1.34543D-01

At iterate  197    f= -4.48391D+00    |proj g|=  4.38517D-01

At iterate  150    f= -3.83299D+00    |proj g|=  3.09610D-01

At iterate  238    f= -4.19182D+00    |proj g|=  9.05973D-01

At iterate   51    f= -6.93992D-01    |proj g|=  2.19036D-01

At iterate   65    f= -2.37429D+00    |proj g|=  4.92874D-01

At iterate   97    f= -3.52017D+00    |proj g|=  6.14037D-01

At iterate  187    f= -3.76546D+00    |proj g|=  3.40712D-01

At iterate  132    f= -5.61390D+00    |proj g|=  8.89152D-01

At iterate  151    f= -3.83341D+00    |proj g|=  1.99569D-01

At iter

KeyboardInterrupt: 